In [1]:
#
# TODO: Agregar data básica del modelo
# * Agregar URLs a publicaciones, imágenes
# ! Los modelos de la base del comparador no están completos, trae unos pocos
# ? SOP: https://docs.google.com/document/d/1zp91sPv0uYKKByQpszNrFQEFBcFzxP0HXX5vgh1azVw/edit?usp=sharing

In [2]:
from google import genai
import os
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR
from db.loader import *
import pandas as pd
pd.set_option('display.max_colwidth', None)

## Instanciar gemini

In [3]:
from processor.gemini_processor import GeminiProcessor
gemini_processor = GeminiProcessor()

### Elegir país de los modelos

In [4]:
pais = "Colombia"

### DB del comparador

In [5]:
df_comparador = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\data\db\bd_comparador_processed.csv")
df_comparador = df_comparador[df_comparador["stock"] == True] # Ignorar modelos que están desconitnuados actualmente

### Base de inventario

In [6]:
db_loader = db_loader(pais)
db_loader.start()
df_columnas_finales = db_loader.data_base

c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\notebooks\..\db\loader.py:18: DtypeWarning: Columns (8,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data_base = pd.read_csv(os.getenv('CO_INVENTORY_PATH'))


### Elección del modelo

In [7]:
df_modelo = df_columnas_finales[df_columnas_finales["code"] == "CO2961-hero-hunk-125-r"]

### Extracción de datos básicos

Se itera entre cada uno de la información disponible para poder adjuntarlo a la base de conocimiento 

In [8]:
from utils.base_knowledge_utils import get_basic_data_model
data_model = get_basic_data_model(df_modelo, pais)

### Comparador

In [9]:
df_top_3_models = df_comparador[df_comparador["url"].str.contains(data_model["code"])].head(3)
df_top_3_models

,url,Views,available_code,stock
4,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO731-tvs-raider-125,2291,"['CO2961-hero-hunk-125-r', 'CO731-tvs-raider-125']",True
19,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2861-tvs-raider-fi-125,1272,"['CO2961-hero-hunk-125-r', 'CO2861-tvs-raider-fi-125']",True
152,https://www.galgo.com/co/comparador/motos/CO2961-hero-hunk-125-r-vs-CO2615-hero-hunk-150-xt,466,"['CO2961-hero-hunk-125-r', 'CO2615-hero-hunk-150-xt']",True


### Extraer info de los top 3 modelos

In [10]:
from utils.comparator_utils import extract_top_3_models_from_comparator
modelos_comparados = extract_top_3_models_from_comparator(df_top_3_models, df_columnas_finales, data_model["code"], pais, debug=True)

Modelos comparados encontrados: 3
1. TVS Raider 125
2. TVS Raider FI 125
3. Hero Hunk 150 XT
Output: 
 [{'marca': 'TVS', 'modelo': 'Raider 125'}, {'marca': 'TVS', 'modelo': 'Raider FI 125'}, {'marca': 'Hero', 'modelo': 'Hunk 150 XT'}]


## Generar data

### Construir datos básicos del modelo

In [11]:
#
# TODO: Agregar imágenes del modelo

In [27]:
basic_data_model_info = f"""
[DATOS BASICOS]
Marca: {data_model["marca"]}
Modelo: {data_model["modelo"]}
Año: {data_model["año"]}
Pais: {data_model["pais"]}
Tipo: {data_model["tipo"]}
URL: {data_model["url"]}
"""

In [28]:
print(basic_data_model_info)


[DATOS BASICOS]
Marca: Hero
Modelo: Hunk 125 R
Año: 2026
Pais: Colombia
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r



### Deep research
Sección exclusiva para generar los deep research usando Gemini

#### 1. Deep research - Información general del modelo

Este research, busca encontrar toda la información relacionada del modelo, según el prompt que se encuentra definido.

In [14]:
from utils.prompt_utils import *

# Leer plantilla del prompt
prompt_from_file = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "knowledge_base_template.md"))

# Reemplazar variables
first_research_prompt = replace_variables(prompt_from_file, {
    "{marca}": data_model["marca"],
    "{modelo}": data_model["modelo"],
    "{año}": str(data_model["año"]),
    "{pais}": pais,
    "{tipo}": data_model["tipo"]
    })

##### Generar research

In [15]:
deep_research = gemini_processor.send_prompt(first_research_prompt)

c:\Users\JTRUJILLO\Documents\Galgo\Scripts\Otros\deep_research_models\src\notebooks\..\processor\gemini_processor.py:9: UserWarning: Interactions usage is experimental and may change in future versions.
  interaction = self.client.interactions.create(


#### 2. Deep research - Modelos mas comparados

Acá se usará la base de datos del comparador de Galgo para poder buscar los top 3 modelos mas comparados y evitarnos un paso más (el buscar cuales son los modelos que mas se les compara)

In [16]:
# Texto de los modelos comparados
modelos_comparados_texto = ""
for i, m in enumerate(modelos_comparados, 1):
    modelos_comparados_texto += f"{i}. {m['marca']} {m['modelo']}\n"

# Leer plantilla del prompt que busca los modelos mas comparados
from utils.prompt_utils import *
prompt_comparaciones = read_prompt_from_file(os.path.join(SRC_DIR, "data", "prompts", "comparisons_template.md"))

# Reemplazar variables
prompt_comparaciones = replace_variables(prompt_comparaciones, {
    "{marca}": data_model["marca"],
    "{modelo}": data_model["modelo"],
    "{pais}": pais,
    "{modelos_comparados}": modelos_comparados_texto.strip()
})

##### Generar research

In [17]:
comparations_research = gemini_processor.send_prompt(prompt_comparaciones)

## Output finales

In [18]:
print(deep_research)

Deep Sentiment Research: **Hero Hunk 125 R (Versión Colombia 2026)**

[SEGMENTO]
Tipo según BD: ['Urbana']
Segmento identificado por usuarios: **Street-Sport de Entrada**. Aunque su mecánica es utilitaria, el usuario la compra casi exclusivamente por su estética de "moto grande" (naked deportiva). Se percibe como el paso intermedio entre una moto de trabajo (ej. Eco Deluxe) y una 160cc real.

[SENTIMIENTO]
En Colombia, la Hunk 125 R es percibida como la **"opción inteligente precio-apariencia"**. El sentimiento general es de satisfacción por obtener una moto que "no parece de 125cc" a un precio inferior a su rival directa (TVS Raider). Sin embargo, hay una resignación consciente: el usuario sabe que sacrifica tecnología de motor (inyección) y refinamiento a cambio de un diseño robusto y una garantía muy extensa (4 años/50.000 km), que genera mucha tranquilidad.

[SENSACIONES]
*   **Estabilidad:** Se siente ligera y ágil en el "trancón" (tráfico pesado), pero nerviosa y "flotante" por e

In [19]:
print(comparations_research)

Basado en el análisis de experiencias de usuarios, foros de motociclismo (Publimotos, Clubes Hero/TVS Colombia) y reviews de propietarios en 2024-2025, aquí tienes la comparación detallada.

[COMPARACIONES]

### 1. Hero Hunk 125 R vs. TVS Raider 125 (Versión Estándar)
**Por qué los usuarios las comparan:**
Son las dos rivales directas por el trono de las "125 deportivas" en Colombia. Los usuarios llegan a esta comparación buscando una moto de entrada que no parezca de trabajo (como una NKD o Boxer), sino que tenga estética de "moto grande" con presupuesto ajustado.

*   **En qué gana el modelo principal (Hero Hunk 125 R):**
    *   **Presencia y Dimensiones:** Es el punto más repetido por los dueños. La Hunk 125 R "aparenta ser una 160cc o 200cc". Tiene un tanque más voluminoso, llantas más anchas (120 atrás vs 100 de la Raider) y un porte más robusto.
    *   **Estabilidad:** Usuarios reportan que, al ser más pesada (aprox. 133kg vs 113kg de la Raider) y tener una suspensión trasera d

## Concatenar researchs

In [29]:
knowledge_base = basic_data_model_info + "\n\n" + deep_research + "\n\n" + comparations_research

In [30]:
print(knowledge_base)


[DATOS BASICOS]
Marca: Hero
Modelo: Hunk 125 R
Año: 2026
Pais: Colombia
Tipo: ['Urbana']
URL: https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r


Deep Sentiment Research: **Hero Hunk 125 R (Versión Colombia 2026)**

[SEGMENTO]
Tipo según BD: ['Urbana']
Segmento identificado por usuarios: **Street-Sport de Entrada**. Aunque su mecánica es utilitaria, el usuario la compra casi exclusivamente por su estética de "moto grande" (naked deportiva). Se percibe como el paso intermedio entre una moto de trabajo (ej. Eco Deluxe) y una 160cc real.

[SENTIMIENTO]
En Colombia, la Hunk 125 R es percibida como la **"opción inteligente precio-apariencia"**. El sentimiento general es de satisfacción por obtener una moto que "no parece de 125cc" a un precio inferior a su rival directa (TVS Raider). Sin embargo, hay una resignación consciente: el usuario sabe que sacrifica tecnología de motor (inyección) y refinamiento a cambio de un diseño robusto y una garantía muy extensa (4 años/50.000 km), que ge

In [ ]:
texto_datos_basicos_modelos_comparados = ""

for modelo in modelos_comparados:
    df_final_comparacion = df_columnas_finales[(df_columnas_finales["Marca"] == modelo["marca"]) & (df_columnas_finales["Modelo"] == modelo["modelo"])]
    for index, row in df_final_comparacion.iterrows():
        marca = row["Marca"]
        modelo = row["Modelo"]
        año = row["Año"]
        pais = pais
        tipo = row["Tipo"]
        code = row["code"]
        url = row["url"]
        texto_datos_basicos_modelos_comparados += f"""
Marca: {marca}
Modelo: {modelo}
Año: {año}
Pais: {pais}
Tipo: {tipo}
URL: {url}
        """

In [ ]:
# #
# # TODO: Agregar imágenes de modelo
# #
# datos_basicos =  f"""
# Datos básicos del modelo:
# Marca: {marca}
# Modelo: {modelo}
# Año: {año}
# Pais: {pais}
# Tipo: {tipo}
# URL: {url}
# """

# otros_datos = f"""
# Datos de los demás modelos comparados:
# {texto_datos_basicos_modelos_comparados}
# """

In [ ]:
print(datos_basicos)

In [ ]:
print(otros_datos)

In [ ]:
final_research_complete += datos_basicos + "\n\n" + otros_datos

In [ ]:
print(final_research_complete)

### Exportar OUTPUT

In [ ]:
with open(f"{SRC_DIR}/data/output_gemini/{code}-{pais}.md", "w", encoding="utf-8") as f:
    f.write(final_output)

### Usando Replicate

In [ ]:
# client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

# output = client.run(
#     ref = "google/gemini-3-pro",
#     input={
#         "prompt": prompt
#     },
# )

# print("\n--- RESPUESTA ---\n")
# if isinstance(output, list):
#     print("".join([str(x) for x in output]))
# else:
#     print(output)